In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"  # specify which GPU(s) to be used

In [2]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, GPTVectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
# from 
from langchain.embeddings import HuggingFaceEmbeddings,HuggingFaceInstructEmbeddings

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
import torch

# device = 'cuda:2,3' if torch.cuda.is_available() else 'cpu'



In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

# tokenizer = AutoTokenizer.from_pretrained("TheBloke/wizardLM-13B-1.0-fp16")
# model = AutoModelForCausalLM.from_pretrained("TheBloke/wizardLM-13B-1.0-fp16",load_in_8bit=True,trust_remote_code=True, device_map=device,)
# embeddings = HuggingFaceInstructEmbeddings(
#     model_name="hkunlp/instructor-large"
# )

/home/jupyter-20523239/.conda/envs/yolov8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-31 12:56:00.122074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 12:56:02.033734: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-31 12:56:02.033855: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 

In [ ]:
# from langchain.llms import HuggingFacePipeline

# # class ModelLLM(LLM):
# generation_pipeline = pipeline(
#     model=model,
#     tokenizer=tokenizer,
# #     return_full_text=True,
#     task="text-generation",
# #     max_length=2048,
#     max_new_tokens=100,
#     temperature=0,
#     top_p = .95,
# #     stopping_criteria=stopping_criteria,
# #     generation_config=generation_config,
#     model_kwargs={
#                     "device_map": "auto", 
#                     "max_length": 512, 
#                     "temperature": 0.01,
#                     "torch_dtype":torch.bfloat16,
#                     }
# )
 

# llm = HuggingFacePipeline(pipeline=generation_pipeline)



In [4]:
# # # # # setup prompts - specific to Camel
# # # # from llama_index.prompts.prompts import SimpleInputPrompt
# # # # question_t5_template = """
# # # #     context: {context}
# # # #     question: {question}
# # # #     answer: 
# # # #     """
# # # # # This will wrap the default prompts that are internal to llama-index
# # # # # taken from https://huggingface.co/Writer/camel-5b-hf
# # # # query_wrapper_prompt = SimpleInputPrompt(
# # # #     question_t5_template
# # # # )
# # # from llama_index.prompts.prompts import SimpleInputPrompt

# # # system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
# # # - StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
# # # - StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
# # # - StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
# # # - StableLM will refuse to participate in anything that could harm a human.
# # # """

# # # # This will wrap the default prompts that are internal to llama-index
# # # query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")
# # from langchain.prompts.chat import (
# #     ChatPromptTemplate,
# #     HumanMessagePromptTemplate,
# #     SystemMessagePromptTemplate,
# # )
# # from llama_index.prompts import Prompt

# # chat_text_qa_msgs = [
# #     SystemMessagePromptTemplate.from_template(
# #         "Always answer the question, even if the context isn't helpful."
# #     ),
# #     HumanMessagePromptTemplate.from_template(
# #         "Context information is below.\n"
# #         "---------------------\n"
# #         "{context_str}\n"
# #         "---------------------\n"
# #         "Given the context information and not prior knowledge, "
# #         "answer the question: {query_str}\n"
# #     ),
# # ]
# # chat_text_qa_msgs_lc = ChatPromptTemplate.from_messages(chat_text_qa_msgs)
# # text_qa_template = Prompt.from_langchain_prompt(chat_text_qa_msgs_lc)

# # # Refine Prompt
# # chat_refine_msgs = [
# #     SystemMessagePromptTemplate.from_template(
# #         "Always answer the question, even if the context isn't helpful."
# #     ),
# #     HumanMessagePromptTemplate.from_template(
# #         "We have the opportunity to refine the original answer "
# #         "(only if needed) with some more context below.\n"
# #         "------------\n"
# #         "{context_msg}\n"
# #         "------------\n"
# #         "Given the new context, refine the original answer to better "
# #         "answer the question: {query_str}. "
# #         "If the context isn't useful, output the original answer again.\n"
# #         "Original Answer: {existing_answer}"
# #     ),
# # ]


# # chat_refine_msgs_lc = ChatPromptTemplate.from_messages(chat_refine_msgs)
# # refine_template = Prompt.from_langchain_prompt(chat_refine_msgs_lc)
# from llama_index import Prompt

# template = (
# #     "We have provided context information below. \n"
# #     "---------------------\n"
# #     "{{context_str}}"
# #     "\n---------------------\n"
#     "Given this information, please answer the question: {query_str}\n"
# )
# qa_template = Prompt(template)
from llama_index.prompts.prompts import SimpleInputPrompt

# query_wrapper_prompt = SimpleInputPrompt(
#     "Below is an instruction that describes a task. "
#     "Write a response that appropriately completes the request.\n\n"
#     "### Context:\n{{context_str}}\n\n"
#     "### Instruction:\n{{query_str}}\n\n### Response:"
# )

system_prompt = """<|SYSTEM|># Custom LM (Alpha version)
- StableLM is a helpful law language model.
- StableLM is more than just an information source, it can compare legal documents based on user context.
- StableLM will refuse to participate in anything that could harm a human.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [5]:
from llama_index.llms import HuggingFaceLLM

# NOTE: the first run of this will download/cache the weights, ~20GB
hf_predictor = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="TheBloke/wizardLM-13B-1.0-fp16",
#     model_name="lmsys/vicuna-13b-v1.3",
    model_name="TheBloke/wizardLM-13B-1.0-fp16",
    device_map='auto',
    tokenizer_kwargs={"max_length": 2048},
    tokenizer_outputs_to_remove=["token_type_ids"],

    model_kwargs={"torch_dtype": torch.bfloat16}
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:51<00:00, 17.15s/it]


In [6]:
from typing import Any, List
from InstructorEmbedding import INSTRUCTOR
from llama_index.embeddings.base import BaseEmbedding
class InstructorEmbeddings(BaseEmbedding):
    def __init__(
    self, 
    instructor_model_name: str = "hkunlp/instructor-large",
        instruction: str = "Represent the document for pasal retrieval",
    **kwargs: Any,) -> None:
        self._model = INSTRUCTOR(instructor_model_name)
        self._instruction = instruction
        
        super().__init__(**kwargs)

    def _get_query_embedding(self, query: str) -> List[float]:
        embeddings = self._model.encode([[self._instruction, query]])
        return embeddings[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        embeddings = self._model.encode([[self._instruction, text]])
        return embeddings[0] 

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self._model.encode([[self._instruction, text] for text in texts])
        return embeddings

In [7]:
import torch
from llama_index import (
    GPTVectorStoreIndex,
    LangchainEmbedding,
    LLMPredictor,
    ServiceContext,
    StorageContext,
    download_loader,
    PromptHelper,
    VectorStoreIndex,
)
# embed_model = LangchainEmbedding(InstructorEmbeddings())

service_context = ServiceContext.from_defaults(chunk_size=512, llm=hf_predictor, embed_model=InstructorEmbeddings(embed_batch_size=2))
# service_context = ServiceContext.from_defaults(embed_model=embed_model)


load INSTRUCTOR_Transformer
max_seq_length  512


In [15]:

doc_1 = SimpleDirectoryReader(input_files=['../Investasi/Peraturan_Presiden_No.10_Tahun_1964.pdf']).load_data()
# print(len(doc_1))
len(doc_1)

3

In [ ]:
doc_2 = SimpleDirectoryReader(input_files=['../Investasi/Perpu Nomor 1 Tahun 2020.pdf']).load_data()
len(doc_2)

In [16]:
from llama_index import ListIndex
doc_1_idx = ListIndex.from_documents(doc_1,service_context=service_context)
# print(len)
len(doc_1_idx.docstore.docs)

5

In [ ]:
from llama_index import ListIndex
doc_2_idx = ListIndex.from_documents(doc_2,service_context=service_context)
# print(len)
# len(doc_1_idx.docstore.docs)

In [ ]:
doc_1_idx.set_index_id("vector_index")
doc_1_idx.storage_context.persist("./storage")

In [ ]:
# rebuild storage context
from llama_index import     load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="storage")
# load index
index = load_index_from_storage(storage_context, index_id="vector_index",service_context=service_context)

In [17]:
query_engine = doc_1_idx.as_query_engine(service_context=service_context)

In [32]:
response = query_engine.query("Is there any direct contradiction in the document?")


In [33]:
response.response

'Based on the additional context provided, there does not appear to be any direct contradiction in the document. The document establishes a committee for the management of state funds for development projects and outlines its responsibilities, including providing recommendations on budgeting, resource allocation, and investment strategies. The document also mentions that the committee will be supported by teams and that its expenses will be covered by the National Planning Development Budget. The document also states that any matters not covered in the decree will be regulated by further regulations issued by the Minister of National Planning. This statement does not contradict any other statements in the document and provides clarity on how any unaddressed matters will be handled. Therefore, the answer to the original question remains the same: Based on the given context information, there does not appear to be any direct contradiction in the document.'

In [12]:
response.response

"Based on the new context provided, there is no direct contradiction in the document. The document discusses the tariff for the central investment agency's central agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment agency's investment"

In [ ]:
response

In [ ]:
response

In [ ]:
response.source_nodes

In [ ]:
doc_2_idx = ListIndex.from_documents(doc_2,service_context=service_context)
# print(len)
len(doc_2_idx.docstore.docs)

In [ ]:
doc_1_engine = doc_1_idx.as_query_engine(service_context=service_context)


In [ ]:
doc_2_engine = doc_2_idx.as_query_engine(service_context=service_context)


In [ ]:
query_engine_tools=[
    QueryEngineTool(
        query_engine=doc_1_engine,
        metadata=ToolMetadata(name='Peraturan Menteri Keuangan No.5 2021',description='Contain Set of Rules/Regulation or Pasal about public service agency rates')
    ),
    QueryEngineTool(
        query_engine=doc_2_engine,
        metadata=ToolMetadata(name='Perpu Nomor 1 Tahun 2020',description='Contain set of rules/regulation or pasal about monetary policy regarding covid-19'))
    ]

In [ ]:
# from llama_index.query_engine import RouterQueryEngine

# query_engine = RouterQueryEngine.from_defaults(
#     query_engine_tools=query_engine_tools
# )

# response = query_engine.query(
#     "In Notion, give me a summary of the product roadmap."
# )
from llama_index.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools,service_context=service_context)


In [ ]:
query_engine = doc_1_idx.as_query_engine()

In [ ]:
qa_template

In [ ]:
response = query_engine.query("Is there any contradiction ? translate the answer in Indonesian")


In [ ]:
response